In [30]:
import os
import random
from pathlib import Path

import pandas as pd
import torch 

import matplotlib.pyplot as plt

## Общие утилиты

In [32]:
IMAGES_PATH = Path('imgs/finetune_gpt/')
DATA_PATH = Path('data/finetune_gpt/')

IMAGES_PATH.mkdir(parents=True, exist_ok=True)
DATA_PATH.mkdir(parents=True, exist_ok=True)
    
def seed_all(seed: int) -> None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

SEED = 42

seed_all(SEED)

# Загрузка данных

[Huggingface](https://huggingface.co/datasets/d0rj/geo-reviews-dataset-2023?row=1)

In [33]:
from datasets import load_dataset

# Загрузка датасета
dataset = load_dataset("d0rj/geo-reviews-dataset-2023", cache_dir=DATA_PATH / 'model_cache')

Generating train split:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [34]:
# Преобразование данных в DataFrame
data_df = pd.DataFrame(dataset['train'])

print("Number of rows and columns in the data set:", data_df.shape)

Number of rows and columns in the data set: (500000, 5)


In [35]:
data_df.head(5)

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [36]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  500000 non-null  object
 1   name_ru  499030 non-null  object
 2   rating   500000 non-null  int64 
 3   rubrics  500000 non-null  object
 4   text     500000 non-null  object
dtypes: int64(1), object(4)
memory usage: 19.1+ MB


## Препроцессинг

In [37]:
work_data = data_df.dropna(subset=['text', 'name_ru', 'rating'])
work_data = work_data.drop_duplicates(subset=['text']).reset_index(drop=True)
work_data['text'] = work_data['text'].str.replace('\\n', ' ')
work_data = work_data[:50000]
work_data['text'][0]

'Московский квартал 2. Шумно : летом по ночам дикие гонки. Грязно : кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. Детские площадки убогие, на большой площади однотипные конструкции. Очень дорогая коммуналка. Часто срабатывает пожарная сигнализация. Жильцы уже не реагируют. В это время, обычно около часа, не работают лифты. Из плюсов - отличная планировка квартир ( Московская 194 ), на мой взгляд. Ремонт от застройщика на 3-. Окна вообще жуть - вместо вентиляции. По соотношению цена/качество - 3.'

In [38]:
work_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  50000 non-null  object
 1   name_ru  50000 non-null  object
 2   rating   50000 non-null  int64 
 3   rubrics  50000 non-null  object
 4   text     50000 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


In [39]:
unique_name_ru = work_data['name_ru'].unique().tolist()

unique_rubrics = work_data['rubrics'].unique().tolist()

# Модель

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from pathlib import Path

class FineTuner:
    def __init__(self, 
                 model_name='ai-forever/rugpt3small_based_on_gpt2', 
                 cache_dir='model_cache',
                 data_path=DATA_PATH):
        self.data_path = Path(data_path)
        
        # Инициализация токенизатора и модели
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name, cache_dir=str(self.data_path / cache_dir))
        self.model = GPT2LMHeadModel.from_pretrained(model_name, cache_dir=str(self.data_path / cache_dir))

    def prepare_data(self, df):
        """
        Подготовка данных для обучения
        """
        # Объединение типа проблемы и исходного текста в одну строку входных данных
        df['input'] = df.apply(
            lambda row: f"<name_ru> {row['name_ru']} <rubrics> {row['rubrics']} <rating> {row['rating']} {self.tokenizer.eos_token}", axis=1
            )
        
        # Добавление к целевому тексту токена окончания строки
        df['output'] = df.apply(lambda row: f" <text> {row['text']} {self.tokenizer.eos_token}", axis=1)
        
        # Подготовка пути для сохранения данных
        dataset_path = self.data_path / 'train_dataset.txt'
        # Запись данных в файл
        with dataset_path.open('w', encoding='utf-8') as file:
            for input_text, target_text in zip(df['input'], df['output']):
                file.write(input_text + ' ' + target_text + '\n')
        return dataset_path

    def fine_tune(self, 
                  dataset_path, 
                  output_name='fine_tuned_model', 
                  num_train_epochs=4, 
                  per_device_train_batch_size=4, 
                  learning_rate=5e-5, 
                  save_steps=10_000):
        """
        Дообучение модели на заданном датасете.
        """
        train_dataset = TextDataset(
            tokenizer=self.tokenizer,
            file_path=str(dataset_path),
            block_size=256
        )

        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer, mlm=False
        )

        training_args = TrainingArguments(
            output_dir=str(self.data_path / output_name),
            overwrite_output_dir=True,
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            save_steps=save_steps,
            learning_rate=learning_rate,
            save_total_limit=2,
            logging_dir=str(self.data_path / 'logs'),
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
        )

        trainer.train()
        # Сохранение обученной модели и токенизатора
        self.model.save_pretrained(str(self.data_path / output_name))
        self.tokenizer.save_pretrained(str(self.data_path / output_name))

In [41]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from pathlib import Path

class TextGenerator:
    def __init__(self, model_name='fine_tuned_model', data_path=DATA_PATH):
        """
        Инициализация модели и токенизатора.
        Загружаем модель и токенизатор из указанного пути.
        """
        model_path = Path(data_path) / model_name
        self.tokenizer = GPT2Tokenizer.from_pretrained(str(model_path))
        self.model = GPT2LMHeadModel.from_pretrained(str(model_path))
        self.model.eval()

    def generate_text(self, 
                    name_ru: str, 
                    rubrics: str, 
                    rating: int,
                    max_length=100, 
                    num_return_sequences=1, 
                    temperature=1.0, 
                    top_k=0, 
                    top_p=1.0, 
                    do_sample=False):
        """
        Генерация текста на основе заданного начального текста (prompt) и параметров.
        
        Параметры:
        - name_ru: Название организации.
        - rubrics: Список рубрик, к которым относится организация.
        - rating: Оценка пользователя.
        - max_length: Максимальная длина сгенерированного текста.
        - num_return_sequences: Количество возвращаемых последовательностей.
        - temperature: Контролирует разнообразие вывода.
        - top_k: Если больше 0, ограничивает количество слов для выборки только k наиболее вероятными словами.
        - top_p: Если меньше 1.0, применяется nucleus sampling.
        - do_sample: Если True, включает случайную выборку для увеличения разнообразия.
        """
        # Формирование prompt
        prompt_text = f"<name_ru> {name_ru} <rubrics> {rubrics} <rating> {rating} {self.tokenizer.eos_token} <text> "
        
        # Кодирование текста в формате, пригодном для модели
        encoded_input = self.tokenizer.encode(prompt_text, return_tensors='pt')
        
        # Генерация текстов
        outputs = self.model.generate(
            encoded_input,
            max_length=max_length + len(encoded_input[0]),
            num_return_sequences=num_return_sequences,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=do_sample,
            no_repeat_ngram_size=2
        )
        
        # Декодирование результатов
        all_texts = [self.tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        
        # Удаление входных данных из текстов
        prompt_length = len(self.tokenizer.decode(encoded_input[0], skip_special_tokens=True))
        trimmed_texts = [text[prompt_length:] for text in all_texts]
        
        # Возврат результатов в виде словаря
        return {
            "full_texts": all_texts,
            "generated_texts": trimmed_texts
        }

# Обучение

In [42]:
finetuner = FineTuner()
dataset_path = finetuner.prepare_data(work_data)
finetuner.fine_tune(dataset_path, output_name='fine_tuned_model_gpt_2')


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

<s>


c:\Users\nymle\Desktop\Project-3\venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


  0%|          | 0/20976 [00:00<?, ?it/s]

{'loss': 2.7908, 'grad_norm': 2.468606948852539, 'learning_rate': 4.880816170861938e-05, 'epoch': 0.1}
{'loss': 2.6852, 'grad_norm': 2.5022025108337402, 'learning_rate': 4.761632341723875e-05, 'epoch': 0.19}
{'loss': 2.6485, 'grad_norm': 2.1006112098693848, 'learning_rate': 4.642448512585813e-05, 'epoch': 0.29}
{'loss': 2.6443, 'grad_norm': 2.417471170425415, 'learning_rate': 4.52326468344775e-05, 'epoch': 0.38}
{'loss': 2.6262, 'grad_norm': 2.042022705078125, 'learning_rate': 4.4040808543096874e-05, 'epoch': 0.48}
{'loss': 2.5911, 'grad_norm': 2.0339229106903076, 'learning_rate': 4.2848970251716244e-05, 'epoch': 0.57}
{'loss': 2.5801, 'grad_norm': 2.005763053894043, 'learning_rate': 4.165713196033562e-05, 'epoch': 0.67}
{'loss': 2.5962, 'grad_norm': 1.8750629425048828, 'learning_rate': 4.0465293668955e-05, 'epoch': 0.76}
{'loss': 2.5823, 'grad_norm': 1.6160539388656616, 'learning_rate': 3.9273455377574375e-05, 'epoch': 0.86}
{'loss': 2.5607, 'grad_norm': 1.962394118309021, 'learning_r

# Предикт

In [43]:
unique_name_ru[:10]

['Московский квартал',
 'Продукты Ермолино',
 'LimeFit',
 'Snow-Express',
 'Студия Beauty Brow',
 'Tele2',
 'У тещи',
 'Smoking Park',
 'Jinju',
 'Kari ГИПЕР']

In [44]:
unique_rubrics[:10]

['Жилой комплекс',
 'Магазин продуктов;Продукты глубокой заморозки;Магазин мяса, колбас',
 'Фитнес-клуб',
 'Пункт проката;Прокат велосипедов;Сапсёрфинг',
 'Салон красоты;Визажисты, стилисты;Салон бровей и ресниц',
 'Оператор сотовой связи;Интернет-провайдер',
 'Кафе',
 'Вейп-шоп;Магазин табака и курительных принадлежностей',
 'Кафе;Кофейня',
 'Магазин обуви;Ювелирный магазин;Детские игрушки и игры']

In [46]:
name_ru = unique_name_ru[1]
rubrics = unique_rubrics[1]
rating = 1

generator = TextGenerator(
    model_name='fine_tuned_model_gpt_2',
    data_path=DATA_PATH
)
generated_texts = generator.generate_text(
    name_ru=name_ru,
    rubrics=rubrics,
    rating=rating,
    max_length=200,
    # num_beams=3 # если несколько последовательностей 
    num_return_sequences=3,
    do_sample=True,
    temperature=0.95,  # Слегка уменьшаем уверенность
    top_k=10,         # Уменьшаем количество рассматриваемых верхних k слов
    top_p=0.95        # Уменьшаем "ядерность" распределения
)
for i, text in enumerate(generated_texts['generated_texts']):
    print(f"Generated Text {i+1}: {text}")

Generated Text 1:  Ужасный магазин, не советую туда ходить, лучше уж на улице купить вкусняшек в дорогу 🤦🏼‍♀️  Не советую никому!!  Свежие овощи и фрукты  не всегда  свежие!  На кассе  очереди  и очереди не из простых 😂  Люди  хотят заработать  денег и купить  вкусненькое  на праздники ,  а не просто купить,  как на  рынке  ✅  Цены высокие  в магазине !  
Generated Text 2:  Очень неприятный магазин. Отвратительное отношение к покупателям. На кассах сидят молодые люди, которые ничего не могут сделать, кроме как спросить. В магазине чисто, товар выкладывают в холодильник, но не проверяют его на месте или нет.  Ужас!  Привозят в магазин замороженные овощи и фрукты, и говорят, что они не свежие. Привезли не те овощи, не то мясо.  
Generated Text 3:  Не покупайте здесь продукты Ермолина. Это не натуральные продукты.  Мясо не свежее, мясо гнилое. Продавцы грубят, что не могут найти покупателей.  
